Complete the following steps before running the notebook:
1. pip install the required packages
    - antlr4-tools
    - antlr4-python3-runtime
2. Extract the given ZIP file in the root of this notebook, so below import statement will work.

**Word of Caution:**
    If you are curious and want to look into the file extracted from zip, be warned that each of that file is more than 10k-15k+ lines long.
And if try to open it using VS Code IDE, then your system might hang for a while. (Learned it the hard way)

     ---------------------------------------- 0.0/622.3 kB ? eta -:--:--
     -------------------------------- ----- 524.3/622.3 kB 4.2 MB/s eta 0:00:01
     -------------------------------------- 622.3/622.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Get

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [54 lines of output]
      running egg_info
      writing lib3\PyYAML.egg-info\PKG-INFO
      writing dependency_links to lib3\PyYAML.egg-info\dependency_links.txt
      writing top-level names to lib3\PyYAML.egg-info\top_level.txt
      Traceback (most recent call last):
        File "c:\Users\GOWTHAMR\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\GOWTHAMR\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\GOWTHAMR\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\

In [7]:
from antlr4 import InputStream, CommonTokenStream, FileStream, ParseTreeWalker
from antlr_plsql.PlSqlLexer import PlSqlLexer
from antlr_plsql.PlSqlParser import PlSqlParser
from antlr_plsql.PlSqlParserListener import PlSqlParserListener

from pathlib import Path

# Hello World

In [8]:
raw = """
create or replace procedure test_proc
is
begin
    dbms_output.put_line('Hello, World!');
end;
"""

In [9]:
lexer = PlSqlLexer(InputStream(raw))
stream = CommonTokenStream(lexer)
parser = PlSqlParser(stream)
tree = parser.sql_script()

In [10]:
print(tree.toStringTree(recog=parser))

(sql_script (unit_statement (create_procedure_body create or replace procedure (procedure_name (identifier (id_expression (regular_id test_proc)))) is (body begin (seq_of_statements (statement (call_statement (routine_name (identifier (id_expression (regular_id dbms_output))) . (id_expression (regular_id put_line))) (function_argument ( (argument (expression (logical_expression (unary_logical_expression (multiset_expression (relational_expression (compound_expression (concatenation (model_expression (unary_expression (atom (constant (quoted_string 'Hello, World!'))))))))))))) )))) ;) end))) ; <EOF>)


# Procedure Extractor

In [11]:
class ProcedureExtractor(PlSqlParserListener):
    def __init__(self):
        self.procedures = {}
    
    def enterCreate_procedure_body(self, ctx: PlSqlParser.Create_procedure_bodyContext):
        procedure_name = ctx.procedure_name().getText()

        self.procedures[procedure_name] = {
            "name": procedure_name,
            "start": ctx.start,
            "stop": ctx.stop,
        }
        

In [13]:
def extract_procedures(file_path: Path) -> dict:

    with file_path.open() as f:
        raw_lines = f.readlines()

    lexer = PlSqlLexer(FileStream(file_path))
    stream = CommonTokenStream(lexer)
    parser = PlSqlParser(stream)
    tree = parser.sql_script()

    extractor = ProcedureExtractor()
    walker = ParseTreeWalker()
    walker.walk(extractor, tree)

    procedures = extractor.procedures

    for procedure_name in procedures:
        start_line = procedures[procedure_name]["start"].line - 1
        stop_line = procedures[procedure_name]["stop"].line + 1

        procedures[procedure_name]["source"] = "".join(raw_lines[start_line:stop_line])
    
    return procedures



In [14]:
# Will Take around a minute or two to run - So be patient
# Also, would suggest to collapse the output of this cell - prints a lot of stuff regardin not able to handle `SERVEROUTPUT`, ignore for now
procedures = extract_procedures(Path("sql_files", "Library.sql"))

line 237:4 extraneous input 'SERVEROUTPUT' expecting {<EOF>, '/', ';'}
line 242:10 extraneous input '&' expecting {'ABORT', 'ABS', 'ABSENT', 'ACCESS', 'ACCESSED', 'ACCESSIBLE', 'ACCOUNT', 'ACL', 'ACOS', 'ACROSS', 'ACTION', 'ACTIONS', 'ACTIVATE', 'ACTIVE', 'ACTIVE_COMPONENT', 'ACTIVE_DATA', 'ACTIVE_FUNCTION', 'ACTIVE_TAG', 'ACTIVITY', 'ADAPTIVE_PLAN', 'ADD', 'ADD_COLUMN', 'ADD_GROUP', 'ADD_MONTHS', 'ADJ_DATE', 'ADMIN', 'ADMINISTER', 'ADMINISTRATOR', 'ADVANCED', 'ADVISE', 'ADVISOR', 'AFD_DISKSTRING', 'AFTER', 'AGENT', 'AGGREGATE', 'A', 'ALIAS', 'ALL', 'ALLOCATE', 'ALLOW', 'ALL_ROWS', 'ALTERNATE', 'ALWAYS', 'ANALYTIC', 'ANALYZE', 'ANCESTOR', 'ANCILLARY', 'AND_EQUAL', 'ANOMALY', 'ANSI_REARCH', 'ANTIJOIN', 'ANY', 'ANYSCHEMA', 'APPEND', 'APPENDCHILDXML', 'APPEND_VALUES', 'APPLICATION', 'APPLY', 'APPROX_COUNT_DISTINCT', 'ARCHIVAL', 'ARCHIVE', 'ARCHIVED', 'ARCHIVELOG', 'ARRAY', 'ASCII', 'ASCIISTR', 'ASIN', 'ASIS', 'ASSEMBLY', 'ASSIGN', 'ASSOCIATE', 'ASYNC', 'ASYNCHRONOUS', 'ATAN2', 'ATAN', 'AT

In [15]:
list(procedures.keys())

['loginCustomer_library',
 'loginEmployee_library',
 'viewItem_library',
 'customerAccount_library',
 'employeeAccount_library',
 'rentItem_library',
 'payFines_library',
 'updateInfoCusto_library',
 'updateInfoEmp_library',
 'addCustomer_library',
 'allMedia_library',
 'handleReturns_library',
 'addBook_library',
 'addVideo_library',
 'removeItem_library',
 'viewCustomer_library']

In [16]:
procedures['loginCustomer_library']

{'name': 'loginCustomer_library',
 'start': <antlr4.Token.CommonToken at 0x160e67ccbd0>,
 'stop': <antlr4.Token.CommonToken at 0x160e606a1d0>,
 'source': "CREATE OR REPLACE PROCEDURE loginCustomer_library(user IN VARCHAR2, pass IN VARCHAR2)\nIS\n  passAux customer.password%TYPE;\n  incorrect_password EXCEPTION;\nBEGIN\n   \n   \n  SELECT password INTO passAux\n  FROM customer\n  WHERE username LIKE user;\n  \n  IF passAux LIKE pass THEN\n    DBMS_OUTPUT.PUT_LINE('User ' || user || ' loging succesfull');\n  ELSE\n    RAISE incorrect_password;\n  END IF;\n  \n  EXCEPTION\n  WHEN no_data_found OR incorrect_password THEN \n       DBMS_OUTPUT.PUT_LINE('Incorrect username or password');\n                                   \nEND;\n\n"}

In [18]:
print(procedures['updateInfoEmp_library']['source'])

CREATE OR REPLACE PROCEDURE updateInfoEmp_library(auxEmployee IN employee.employeeid%TYPE, pNumber NUMBER, address VARCHAR2, newPass VARCHAR2, newPayCheck NUMBER,
newBranch VARCHAR2)
IS
BEGIN
  UPDATE employee
  SET phone = pNumber, customeraddress = address, password = newPass, paycheck = auxEmployee, branchname = newBranch
  WHERE employeeid = auxEmployee;
END;


